In [1]:
import os
import csv
import utils
import pandas as pd
import spotipy

# Extract Saved Tracks

In [2]:
def update_saved_tracks(path = 'output/saved_tracks.csv'):
    '''
    Updates saved tracks data by connecting to Spotify API to retrieve a user's saved tracks data.
    Collects Spotify audio features as well as computing rich audio features on each track.
    '''
    
    if os.path.exists(path):
        
        print('Recovering tracks...')
    
        saved_tracks = {}
        with open(path) as csvFile:
            csvReader = csv.DictReader(csvFile)
            for rows in csvReader:
                track_key = rows['track_key']
                saved_tracks[track_key] = rows
        number_existing_saved_tracks = len(saved_tracks)
        
        print(f'Recovered {number_existing_saved_tracks} saved tracks')
    
    else: 
        saved_tracks = {}
    
    print('Connecting to Spotify to aquire new saved tracks now...')
    
    token = utils.get_token()
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks() 
    
    counter = 1
    failed_counter = 0
    track_counter = 0
    ignition = True
    
    
    while ignition:
        if counter % 5 == 0:
            token = utils.get_token()
            sp = spotipy.Spotify(auth = token)
            
        for item in results['items']:
            track = item['track']
            track_name = track['name']
            artist_name = track['artists'][0]['name']
            track_key = track_name + '___' + artist_name

            if track_key in saved_tracks:
                if saved_tracks.get(track_key)['added_at'] == item['added_at']:
                    ignition = False
                    print(f'Reached overlap in saved songs: {track_name} by {artist_name}')
                    break
        
            try:
                track_dictionary = utils.create_track_dictionary(track, token)
                track_dictionary['added_at'] = item['added_at']
                rich_feats = utils.collect_rich_features(track_dictionary['preview_url'])
                track_dictionary.update(rich_feats)
                saved_tracks[track_key] = track_dictionary
                track_counter += 1
            except:
                failed_counter += 1
                
                
        ignition = ignition & bool(results['next'])
        if ignition:
            results = sp.next(results)
        counter += 1
    
    number_saved_tracks = len(saved_tracks)
    print('\n')  
    print(f'You added {track_counter} new tracks to your saved tracks since the last update!')
    print(f'Your saved tracks are now comprised of {number_saved_tracks} unique tracks')
    print(f'Overall, {failed_counter} saved tracks were unable to be identified')
    print(f'{number_saved_tracks} out of {number_saved_tracks + failed_counter} saved tracks have been identified')
    
    print('\n')   
    percent_identified = (number_saved_tracks) / (number_saved_tracks + failed_counter)
    percent_identified = round(percent_identified * 100, 2)
    print(f'{percent_identified}% of saved tracks have been identified')
    
    saved_tracks_frame = pd.DataFrame.from_dict(saved_tracks, orient = 'index')
    saved_tracks_frame.to_csv('output/saved_tracks.csv', index = False)

In [3]:
update_saved_tracks()

Recovering tracks...
Recovered 11038 saved tracks
Connecting to Spotify to aquire new saved tracks now...
Reached overlap in saved songs: Petu - Radio Edit by Palms Trax


You added 51 new tracks to your saved tracks since the last update!
Your saved tracks are now comprised of 11083 unique tracks
Overall, 0 saved tracks were unable to be identified
11083 out of 11083 saved tracks have been identified


100.0% of saved tracks have been identified
